## Récupération et préparation des données


In [1]:
from fredapi import Fred
import matplotlib.pyplot as plt

import numpy as np
from numpy.linalg import svd

import pandas as pd
from pathlib import Path
import re
import requests

import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from typing import List, Dict
import os


from skfin.dataloaders.cache import CacheManager


/Users/alexd/PycharmProjects/ML for PM/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_fred_md():
    # Dataset principal
    url_data = "https://raw.githubusercontent.com/lxsd111/ML_PM/main/nbs/data/2025-09-MD.csv"
    df = pd.read_csv(url_data, index_col=0)

    # Conversion de l'index en dates si possible
    try:
        df.index = pd.to_datetime(df.index)
    except:
        pass
    
    return df

def load_transform_codes():
    url_codes = "https://raw.githubusercontent.com/lxsd111/ML_PM/main/nbs/data/FRED-MD_updated_appendix.csv"
    # cp1252 (a.k.a. windows-1252) fixes the 0x92 decode error
    return pd.read_csv(url_codes, encoding="cp1252")  # or encoding="latin1"


# Chargement des deux fichiers
df = load_fred_md()
transform_codes = load_transform_codes()

df.head(), transform_codes.head()

(                 RPI  W875RX1  DPCERA3M086SBEA    CMRMTSPLx      RETAILx  \
 sasdate                                                                    
 Transform:     5.000      5.0            5.000       5.0000      5.00000   
 1/1/1959    2583.560   2426.0           15.188  276676.8154  17689.23968   
 2/1/1959    2593.596   2434.8           15.346  278713.9773  17819.01912   
 3/1/1959    2610.396   2452.7           15.491  277775.2539  17967.91336   
 4/1/1959    2627.446   2470.0           15.435  283362.7075  17978.97983   
 
              INDPRO  IPFPNSS  IPFINAL  IPCONGD  IPDCONGD  ...  \
 sasdate                                                   ...   
 Transform:   5.0000   5.0000   5.0000   5.0000    5.0000  ...   
 1/1/1959    21.9616  23.3868  22.2620  31.6664   18.9498  ...   
 2/1/1959    22.3917  23.7024  22.4549  31.8987   19.0492  ...   
 3/1/1959    22.7142  23.8459  22.5651  31.8987   19.4223  ...   
 4/1/1959    23.1981  24.1903  22.8957  32.4019   19.5466  ... 

In [3]:
CACHE_DIR = Path("data")
CACHE_DIR.mkdir(parents=True, exist_ok=True)

cm = CacheManager(cache_dir=CACHE_DIR)

FILENAME = CACHE_DIR / "FRED_MD.parquet"

# Sauvegarde correcte des deux DataFrames
cm.save_to_cache(df, FILENAME)

In [4]:
print("\nTypes de données :")
print(df.dtypes)

# Vérification des doublons
n_duplicates = df.duplicated().sum()
print(f"\nNombre de lignes dupliquées : {n_duplicates}")
 
# Vérification des NaN
print("\nValeurs manquantes par colonne :")
missing = df.isna().sum()
display(missing[missing > 0])


Types de données :
RPI                float64
W875RX1            float64
DPCERA3M086SBEA    float64
CMRMTSPLx          float64
RETAILx            float64
                    ...   
UMCSENTx           float64
DTCOLNVHFNM        float64
DTCTHFNM           float64
INVEST             float64
VIXCLSx            float64
Length: 126, dtype: object

Nombre de lignes dupliquées : 0

Valeurs manquantes par colonne :


CMRMTSPLx          1
PERMIT            12
PERMITNE          12
PERMITMW          12
PERMITS           12
PERMITW           12
ACOGNO           398
ANDENOx          109
BUSINVx            1
ISRATIOx           1
NONREVSL           1
CONSPI             1
S&P div yield      2
S&P PE ratio       1
CP3Mx              1
COMPAPFFx          1
TWEXAFEGSMTHx    168
UMCSENTx         154
DTCOLNVHFNM        1
DTCTHFNM           1
VIXCLSx           42
dtype: int64

Dans un premier temps, nous avons inspecté la base de données afin d’identifier la présence éventuelle de doublons ainsi que le taux de valeurs manquantes par série. 

Pour garantir un panel cohérent et exploitable en analyse factorielle et en apprentissage statistique, nous avons effectué les étapes suivantes :
- Restriction de l’échantillon à la période 1962-05 à 2025-07 : on élimine les observations trop anciennes ou trop récentes où plusieurs variables ne sont pas encore disponibles. Cela réduit l’impact des fortes valeurs manquantes en début et fin de série
- Suppression de quatre variables qui contiennent trop peu d’observations. 

In [5]:
# 1) Conversion obligatoire de l’index en datetime
df.index = pd.to_datetime(df.index, errors="coerce")
df = df.sort_index()

cols_to_drop = ["ACOGNO", "TWEXAFEGSMTHx", "UMCSENTx", "ANDENOx"]
df = df.drop(columns=cols_to_drop, errors="ignore")
import pandas as pd
import numpy as np

df = df.loc[df.index >= "1962-05-01"].sort_index()


def trailing_nans(s: pd.Series) -> int:
    """Compte le nombre de NaN consécutifs à la fin de la série."""
    if not s.isna().any():
        return 0
    return int(np.argmax(~s[::-1].isna().values)) if s.isna().any() else 0

nan_trails = {col: trailing_nans(df[col]) for col in df.columns}
nan_trails = {k: v for k, v in nan_trails.items() if v > 0}

print("Variables avec des NaN en fin de série :")
for var, n in nan_trails.items():
    print(f"  {var:<25} → {n} observations manquantes en bas")


# Décale de 'n' mois les séries ayant n NaN en fin de période
for col, lag in nan_trails.items():
    if lag > 0:
        df[col] = df[col].shift(lag)
        print(f"Décalage appliqué sur {col:<25} : {lag} mois")
        
df = df.dropna(how="any")

print(f"\nTaille finale du dataset : {df.shape[0]} observations, {df.shape[1]} variables")


Variables avec des NaN en fin de série :
  CMRMTSPLx                 → 1 observations manquantes en bas
  BUSINVx                   → 1 observations manquantes en bas
  ISRATIOx                  → 1 observations manquantes en bas
  NONREVSL                  → 1 observations manquantes en bas
  CONSPI                    → 1 observations manquantes en bas
  S&P div yield             → 2 observations manquantes en bas
  S&P PE ratio              → 1 observations manquantes en bas
  DTCOLNVHFNM               → 1 observations manquantes en bas
  DTCTHFNM                  → 1 observations manquantes en bas
Décalage appliqué sur CMRMTSPLx                 : 1 mois
Décalage appliqué sur BUSINVx                   : 1 mois
Décalage appliqué sur ISRATIOx                  : 1 mois
Décalage appliqué sur NONREVSL                  : 1 mois
Décalage appliqué sur CONSPI                    : 1 mois
Décalage appliqué sur S&P div yield             : 2 mois
Décalage appliqué sur S&P PE ratio              : 

La variable VIX présente deux valeurs manquantes, que nous pouvons pour le moment nous permetre de conserver puisque cette variable ne subit pas de transformation (t-code 1), ces deux variables disparaitront lorsque nous enleverons les Na apprès la transformation des autres varibales. 

A part le VIX, il ne reste plus que trois valeurs manquantes dans le dataframe, nous les remplassons par la dernière valeur connue.

In [6]:
# Vérification des NaN
print("\nValeurs manquantes par colonne :")
missing = df.isna().sum()
display(missing[missing > 0])


Valeurs manquantes par colonne :


Series([], dtype: int64)

In [7]:
print("\nInfo générale :")
df.info()
 
print("\nStatistiques descriptives :")
display(df.describe().T)
display(df.tail())
display(df.head())


Info générale :
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 757 entries, 1962-07-01 to 2025-08-01
Columns: 122 entries, RPI to VIXCLSx
dtypes: float64(98), int64(24)
memory usage: 727.4 KB

Statistiques descriptives :


,count,mean,std,min,25%,50%,75%,max
RPI,757.0,10060.306425,5029.911155,2960.00800,5732.6670,8840.9180,1.387718e+04,2.291842e+04
W875RX1,757.0,8550.161691,3972.858089,2769.40000,5093.2000,7597.7000,1.171940e+04,1.673990e+04
DPCERA3M086SBEA,757.0,59.615032,30.551716,17.13500,33.0680,52.5900,8.516200e+01,1.248050e+02
CMRMTSPLx,757.0,860467.820772,377680.022390,291713.95990,524334.1780,776545.9350,1.205143e+06,1.565742e+06
RETAILx,757.0,234678.170455,190988.945647,19741.56573,66280.0710,181077.0000,3.619790e+05,7.320100e+05
...,...,...,...,...,...,...,...,...
CES3000000008,757.0,12.449696,7.152718,2.27000,5.9300,11.9000,1.841000e+01,2.903000e+01
DTCOLNVHFNM,757.0,150208.919894,146600.569198,7242.00000,14383.0000,88311.6500,2.740416e+05,5.597089e+05
DTCTHFNM,757.0,378426.503316,341198.823676,16228.00000,38156.0000,213063.0900,7.333905e+05,9.493062e+05
INVEST,757.0,1449.044971,1547.258399,87.78730,236.1659,885.6527,2.289933e+03,5.846708e+03


,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
sasdate,,,,,,,,,,,,,,,,,,,,,
2025-04-01,20698.762,16739.9,123.748,1563633.0,721789.0,103.6224,101.3671,101.1112,101.6979,101.0930,...,106.066,119.658,131.767,32.22,36.96,28.78,554966.69,938997.88,5500.0706,32.5116
2025-05-01,20581.743,16703.7,123.575,1555663.0,716101.0,103.6570,101.4038,101.1856,101.5808,102.5239,...,106.088,119.780,132.071,32.31,37.08,28.87,554180.13,940362.47,5526.3170,20.3620
2025-06-01,20575.971,16664.7,123.894,1550522.0,723033.0,104.2115,101.7271,101.5445,101.9628,101.0181,...,106.590,120.208,132.386,32.40,37.23,28.94,551829.26,938763.49,5555.2136,18.3246
2025-07-01,20625.729,16718.9,124.370,1556845.0,727414.0,103.8194,101.4573,101.4961,101.7345,100.6594,...,106.471,120.036,132.778,32.47,37.28,29.01,549682.41,937344.92,5585.9624,16.4718
2025-08-01,20646.475,16726.7,124.805,1565742.0,732010.0,103.9203,101.5916,101.7216,102.1138,101.2996,...,106.392,120.329,133.209,32.56,37.47,29.03,547389.12,934567.19,5624.3483,15.7545


,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
sasdate,,,,,,,,,,,,,,,,,,,,,
1962-07-01,2960.008,2769.4,17.135,293308.8034,19741.56573,25.4023,27.3468,26.2846,37.1248,23.2271,...,64.096,18.645,10.979,2.39,2.90,2.27,7242.0,16228.0,87.7873,19.5715
1962-08-01,2964.773,2772.5,17.183,291713.9599,19864.30287,25.4292,27.3468,26.2295,36.6989,22.9038,...,64.104,18.696,10.989,2.40,2.90,2.28,7344.0,16453.0,87.8828,15.7942
1962-09-01,2966.166,2775.8,17.375,298514.3889,19923.65936,25.5905,27.5190,26.3397,36.9312,23.2023,...,64.159,18.913,10.997,2.40,2.91,2.28,7442.0,16685.0,88.2080,18.3148
1962-10-01,2981.840,2785.0,17.289,293815.9271,20236.53848,25.6174,27.4329,26.3673,36.8538,23.3018,...,63.860,18.865,11.010,2.40,2.91,2.29,7407.0,16758.0,89.7636,25.9671
1962-11-01,2992.828,2798.3,17.519,299684.9733,20342.17291,25.7249,27.6050,26.4775,37.0861,23.3266,...,63.883,18.862,11.027,2.41,2.93,2.29,7453.0,16883.0,89.7457,16.7658


In [8]:
# Harmoniser noms des colonnes du df
df.columns = df.columns.str.upper().str.replace("X$", "", regex=True)

# Harmoniser noms de la colonne 'fred'
transform_codes['fred'] = transform_codes['fred'].str.upper()

# Mettre 'fred' comme index pour lookup direct
transform_codes = transform_codes.set_index('fred')

df_transformed = pd.DataFrame(index=df.index)


for col in df.columns:
    if col not in transform_codes.index:

        s = df[col]
    else:
        code = int(transform_codes.loc[col, "tcode"])
        s = df[col]

        if code == 1:
            s = s
        elif code == 2:
            s = s.diff()
        elif code == 3:
            s = s.diff().diff()
        elif code == 4:
            # log niveau → attention aux valeurs <= 0
            s = s.where(s > 0).apply(np.log)
        elif code == 5:
            s = s.where(s > 0).apply(np.log).diff()
        elif code == 6:
            s = s.where(s > 0).apply(np.log).diff().diff()
        else:
        # codes atypiques (ex. 7) → on laisse tel quel
            s = s

    df_transformed[col] = s    

# Aperçu
df_transformed.head()

/var/folders/d3/08gxms8j7zq9sxn5lh0v48s80000gn/T/ipykernel_22407/1063616935.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transformed[col] = s
/var/folders/d3/08gxms8j7zq9sxn5lh0v48s80000gn/T/ipykernel_22407/1063616935.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transformed[col] = s
/var/folders/d3/08gxms8j7zq9sxn5lh0v48s80000gn/T/ipykernel_22407/1063616935.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.

,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPL,RETAIL,INDPRO,IPFPNSS,IPFINAL,IPCONGD,IPDCONGD,...,DDURRG3M086SBEA,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLS
sasdate,,,,,,,,,,,,,,,,,,,,,
1962-07-01,NaN,NaN,NaN,293308.8034,19741.56573,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.5715
1962-08-01,0.001608,0.001119,0.002797,291713.9599,19864.30287,0.001058,0.000000,-0.002098,-0.011538,-0.014017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.7942
1962-09-01,0.000470,0.001190,0.011112,298514.3889,19923.65936,0.006323,0.006277,0.004193,0.006310,0.012949,...,0.000733,0.008808,-0.000183,-0.004175,0.003442,-0.004396,-0.000730,0.000233,0.002606,18.3148
1962-10-01,0.005270,0.003309,-0.004962,293815.9271,20236.53848,0.001051,-0.003134,0.001047,-0.002098,0.004279,...,-0.005529,-0.014081,0.000454,0.000000,-0.003442,0.004376,-0.017970,-0.009637,0.013788,25.9671
1962-11-01,0.003678,0.004764,0.013216,299684.9733,20342.17291,0.004188,0.006254,0.004171,0.006284,0.001064,...,0.005031,0.002382,0.000361,0.004158,0.006849,-0.004376,0.010905,0.003066,-0.017681,16.7658


In [9]:
# 2) Coupe du dataset
df_transformed = df_transformed.loc[df_transformed.index >= "1962-07-01"]

# Vérification des NaN
print("\nValeurs manquantes par colonne :")
missing = df_transformed.isna().sum()
display(missing[missing > 0])

print(df_transformed.info())


Valeurs manquantes par colonne :


RPI                1
W875RX1            1
DPCERA3M086SBEA    1
INDPRO             1
IPFPNSS            1
                  ..
CES2000000008      2
CES3000000008      2
DTCOLNVHFNM        2
DTCTHFNM           2
INVEST             2
Length: 87, dtype: int64

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 757 entries, 1962-07-01 to 2025-08-01
Columns: 122 entries, RPI to VIXCLS
dtypes: float64(121), int64(1)
memory usage: 727.4 KB
None


In [ ]:
split_date = "2016-01-01"
train = df_transformed.loc[:split_date].copy()
test  = df_transformed.loc[split_date:].copy()

Une normalisation des données est nécessaire avant une Analyse en Composante Principale. Le test set est standardisé sur les moments du train set.